In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import imdb
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Conv2DTranspose,MaxPooling1D, Embedding,Input, Conv1D, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from tensorflow.keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils
from keras.preprocessing import sequence

In [2]:
top_words = 1000
# Now we split our data-set into training and test data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


model = Sequential()
model.add(Embedding(1000, 32, input_length=max_words))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
# model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu',name="d1"))

model.add(Dense(1, activation='sigmoid', name="d2"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           32000     
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 64)           6208      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
d1 (Dense)                   (None, 256)               4096256   
_________________________________________________________________
d2 (Dense)                   (None, 1)                 2

In [4]:
# Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
196/196 [==============================] - 35s 15ms/step - loss: 0.5490 - accuracy: 0.6683 - val_loss: 0.4303 - val_accuracy: 0.8078
Epoch 2/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3117 - accuracy: 0.8681 - val_loss: 0.3042 - val_accuracy: 0.8712
Epoch 3/10
196/196 [==============================] - 3s 17ms/step - loss: 0.2772 - accuracy: 0.8863 - val_loss: 0.2933 - val_accuracy: 0.8768
Epoch 4/10
196/196 [==============================] - 3s 14ms/step - loss: 0.2579 - accuracy: 0.8945 - val_loss: 0.2896 - val_accuracy: 0.8784
Epoch 5/10
196/196 [==============================] - 3s 14ms/step - loss: 0.2360 - accuracy: 0.9044 - val_loss: 0.2935 - val_accuracy: 0.8780
Epoch 6/10
196/196 [==============================] - 3s 14ms/step - loss: 0.2138 - accuracy: 0.9147 - val_loss: 0.2992 - val_accuracy: 0.8769
Epoch 7/10
196/196 [==============================] - 3s 14ms/step - loss: 0.1841 - accuracy: 0.9281 - val_loss: 0.3168 - val_accuracy: 0.871

In [5]:
scores = model.evaluate(X_test, y_test, verbose=1)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 2s 3ms/step - loss: 0.4679 - accuracy: 0.8649
Accuracy: 86.49%


In [6]:
teacher_WO_Softmax = Model(model.input, model.get_layer('d1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
# val_dense = teacher_WO_Softmax.predict(X_val)


s1Train=train_dense[:,:32]
s2Train=train_dense[:,32:64]
s3Train=train_dense[:,64:96]
s4Train=train_dense[:,96:128]

s5Train=train_dense[:,128:160]
s6Train=train_dense[:,160:192]
s7Train=train_dense[:,192:224]
s8Train=train_dense[:,224:]


In [7]:
s7Train.shape

(25000, 32)

In [8]:

def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=(500,))
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile('adam',loss=['binary_crossentropy','mse'],loss_weights=[0.5,0.5],metrics=['accuracy'])

    return gan

In [15]:
def build_sdiscriminator():
    
    input2 = Input(shape=(32,),name='input')
    inp=Dense(128)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    conv3 = Dense(128,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    b_n=Dropout(0.25)(b_n)
    conv3 = Dense(256,activation='relu')(b_n)
    b_n = BatchNormalization()(conv3)
    conv4 = Dense(256,activation='relu')(b_n)
    b_n = BatchNormalization()(conv4)
    dense = Dense(1,activation='sigmoid')(b_n)
    output2=Dense(32,activation='relu')(b_n)
    disc = Model(input2,[dense,output2])          
    disc.compile(Adam(lr=0.0002,beta_1=0.5),loss=['binary_crossentropy','mse',],metrics=['accuracy'])


    return disc

In [16]:
def define_model(name):
  model = Sequential()
  model.add(tf.keras.layers.Embedding(1000, 10, input_length=500,name=name))
  model.add(Dropout(0.2))
  model.add(Conv1D(32, 3,strides=3,
                  activation='relu'))
  model.add(MaxPooling1D())
  model.add(Conv1D(32, 3,strides=3,
                  activation='relu'))
  model.add(MaxPooling1D())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(32, activation='relu',name=name+"req"))
	# compile model
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  return model

In [17]:
define_model("s1").summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Embedding)               (None, 500, 10)           10000     
_________________________________________________________________
dropout_43 (Dropout)         (None, 500, 10)           0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 166, 32)           992       
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 83, 32)            0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 27, 32)            3104      
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 13, 32)            0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 416)             

In [18]:
def training(generator,discriminator,gan,features,epo=20):
    BATCH_SIZE = 128
    # discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
            
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [19]:
s1=define_model("s1")
d1 = build_sdiscriminator()
gan1 = build_gan(s1,d1)

s2=define_model("s2")
d2 = build_sdiscriminator()
gan2 = build_gan(s2,d2)

s3=define_model("s3")
d3 = build_sdiscriminator()
gan3 = build_gan(s3,d3)

s4=define_model("s4")
d4 = build_sdiscriminator()
gan4 = build_gan(s4,d4)

s5=define_model("s5")
d5 = build_sdiscriminator()
gan5 = build_gan(s5,d5)

s6=define_model("s6")
d6 = build_sdiscriminator()
gan6 = build_gan(s6,d6)

s7=define_model("s7")
d7 = build_sdiscriminator()
gan7 = build_gan(s7,d7)

s8=define_model("s8")
d8 = build_sdiscriminator()
gan8 = build_gan(s8,d8)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [20]:
s1=training(s1,d1,gan1,s1Train,20)
s2=training(s2,d2,gan2,s2Train,20)

195/195 [==============================] - 9s 35ms/step
Epoch: 1 | Discriminator Loss: 0.966913 | Generator Loss: 0.430333 | 
195/195 [==============================] - 7s 35ms/step
Epoch: 2 | Discriminator Loss: 0.786830 | Generator Loss: 0.380317 | 
195/195 [==============================] - 7s 37ms/step
Epoch: 3 | Discriminator Loss: 0.761509 | Generator Loss: 0.388919 | 
195/195 [==============================] - 7s 37ms/step
Epoch: 4 | Discriminator Loss: 0.752486 | Generator Loss: 0.392527 | 
195/195 [==============================] - 7s 36ms/step
Epoch: 5 | Discriminator Loss: 0.750567 | Generator Loss: 0.393276 | 
195/195 [==============================] - 7s 36ms/step
Epoch: 6 | Discriminator Loss: 0.747448 | Generator Loss: 0.375986 | 
195/195 [==============================] - 7s 36ms/step
Epoch: 7 | Discriminator Loss: 0.743312 | Generator Loss: 0.372637 | 
195/195 [==============================] - 7s 35ms/step
Epoch: 8 | Discriminator Loss: 0.744151 | Generator Loss: 0.36

In [21]:
s3=training(s3,d3,gan3,s3Train,20)
s4=training(s4,d4,gan4,s4Train,20)

195/195 [==============================] - 9s 35ms/step
Epoch: 1 | Discriminator Loss: 1.225014 | Generator Loss: 0.647740 | 
195/195 [==============================] - 7s 35ms/step
Epoch: 2 | Discriminator Loss: 0.918716 | Generator Loss: 0.515352 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 3 | Discriminator Loss: 0.862227 | Generator Loss: 0.494061 | 
195/195 [==============================] - 7s 35ms/step
Epoch: 4 | Discriminator Loss: 0.849639 | Generator Loss: 0.483912 | 
195/195 [==============================] - 7s 35ms/step
Epoch: 5 | Discriminator Loss: 0.836816 | Generator Loss: 0.480021 | 
195/195 [==============================] - 7s 35ms/step
Epoch: 6 | Discriminator Loss: 0.826431 | Generator Loss: 0.469242 | 
195/195 [==============================] - 7s 36ms/step
Epoch: 7 | Discriminator Loss: 0.821225 | Generator Loss: 0.471658 | 
195/195 [==============================] - 7s 36ms/step
Epoch: 8 | Discriminator Loss: 0.817748 | Generator Loss: 0.46

In [22]:
s5=training(s5,d5,gan5,s5Train,10)
s6=training(s6,d6,gan6,s6Train,20)
s7=training(s7,d7,gan7,s7Train,15)
s8=training(s8,d8,gan8,s8Train,20)

195/195 [==============================] - 9s 35ms/step
Epoch: 1 | Discriminator Loss: 1.290351 | Generator Loss: 0.690168 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 2 | Discriminator Loss: 0.937171 | Generator Loss: 0.512054 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 3 | Discriminator Loss: 0.895687 | Generator Loss: 0.462172 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 4 | Discriminator Loss: 0.881445 | Generator Loss: 0.453494 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 5 | Discriminator Loss: 0.867747 | Generator Loss: 0.469739 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 6 | Discriminator Loss: 0.861812 | Generator Loss: 0.501703 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 7 | Discriminator Loss: 0.858483 | Generator Loss: 0.495308 | 
195/195 [==============================] - 7s 34ms/step
Epoch: 8 | Discriminator Loss: 0.855128 | Generator Loss: 0.50

In [23]:
o1=s1.get_layer("s1req").output
o2=s2.get_layer("s2req").output
o3=s3.get_layer("s3req").output
o4=s4.get_layer("s4req").output
o5=s5.get_layer("s5req").output
o6=s6.get_layer("s6req").output
o7=s7.get_layer("s7req").output
o8=s8.get_layer("s8req").output

out = tensorflow.keras.layers.concatenate([o1,o2,o3,o4,o5,o6,o7,o8])

output=Activation('relu')(out)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)

mm8=Model([s1.get_layer("s1").input,
           s2.get_layer("s2").input,
           s3.get_layer("s3").input,
           s4.get_layer("s4").input,
           s5.get_layer("s5").input,
           s6.get_layer("s6").input,
           s7.get_layer("s7").input,
           s8.get_layer("s8").input
           ],
          output3)
my_weights=model.get_layer('d2').get_weights()
mm8.get_layer('d11').set_weights(my_weights)

In [24]:
i=0
for l in mm8.layers[:len(mm8.layers)-1]:
    l.trainable=False

In [25]:
adam = Adam(0.0002,0.5,0.999)

mm8.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [26]:
batch_size = 128
mm8_history=mm8.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test], y_test))

Epoch 1/10
196/196 [==============================] - 5s 16ms/step - loss: 2.9127 - accuracy: 0.5370 - val_loss: 0.8479 - val_accuracy: 0.5956
Epoch 2/10
196/196 [==============================] - 3s 14ms/step - loss: 2.1942 - accuracy: 0.5837 - val_loss: 0.5448 - val_accuracy: 0.7275
Epoch 3/10
196/196 [==============================] - 3s 14ms/step - loss: 1.7411 - accuracy: 0.6170 - val_loss: 0.4312 - val_accuracy: 0.7980
Epoch 4/10
196/196 [==============================] - 3s 14ms/step - loss: 1.4039 - accuracy: 0.6431 - val_loss: 0.3991 - val_accuracy: 0.8216
Epoch 5/10
196/196 [==============================] - 3s 14ms/step - loss: 1.1674 - accuracy: 0.6777 - val_loss: 0.3974 - val_accuracy: 0.8289
Epoch 6/10
196/196 [==============================] - 3s 14ms/step - loss: 0.9742 - accuracy: 0.7075 - val_loss: 0.4097 - val_accuracy: 0.8296
Epoch 7/10
196/196 [==============================] - 3s 14ms/step - loss: 0.8536 - accuracy: 0.7298 - val_loss: 0.4074 - val_accuracy: 0.8334

In [27]:
# 1 student evaluate model
_, acc = mm8.evaluate([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test], y_test, verbose=1)
print('> %.2f' % (acc * 100.0))

782/782 [==============================] - 5s 6ms/step - loss: 0.4132 - accuracy: 0.8354
> 83.54


In [28]:
mm8.summary()

Model: "model_37"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s3_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s4_input (InputLayer)           [(None, 500)]        0                                            
___________________________________________________________________________________________

## FeedForward Method


In [30]:
s1= define_model("s1")
s2= define_model("s2")
s3= define_model("s3")
s4= define_model("s4")

In [31]:
batch_size = 128
mm_history=s1.fit(X_train, s1Train, batch_size=batch_size, epochs=10,verbose=1)
mm2_history=s2.fit(X_train, s2Train, batch_size=batch_size, epochs=10,verbose=1)


Epoch 1/10
196/196 [==============================] - 1s 4ms/step - loss: 0.1781 - accuracy: 0.3070
Epoch 2/10
196/196 [==============================] - 1s 4ms/step - loss: 0.1089 - accuracy: 0.3756
Epoch 3/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0897 - accuracy: 0.3847
Epoch 4/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0832 - accuracy: 0.4060
Epoch 5/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0796 - accuracy: 0.4230
Epoch 6/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0765 - accuracy: 0.4410
Epoch 7/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0748 - accuracy: 0.4577
Epoch 8/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0733 - accuracy: 0.4594
Epoch 9/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0714 - accuracy: 0.4679
Epoch 10/10
196/196 [==============================] - 1s 4ms/step - loss: 0.0706 - accuracy: 0.4700

In [32]:
mm3_history=s3.fit(X_train, s3Train, batch_size=batch_size, epochs=10,verbose=1)
mm4_history=s4.fit(X_train, s4Train, batch_size=batch_size, epochs=10,verbose=1)

Epoch 1/10
196/196 [==============================] - 2s 4ms/step - loss: 0.6460 - accuracy: 0.3615
Epoch 2/10
196/196 [==============================] - 1s 4ms/step - loss: 0.4024 - accuracy: 0.4386
Epoch 3/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3115 - accuracy: 0.5906
Epoch 4/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2844 - accuracy: 0.6348
Epoch 5/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2721 - accuracy: 0.6544
Epoch 6/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2628 - accuracy: 0.6678
Epoch 7/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2630 - accuracy: 0.6783
Epoch 8/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2539 - accuracy: 0.6910
Epoch 9/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2499 - accuracy: 0.6940
Epoch 10/10
196/196 [==============================] - 1s 4ms/step - loss: 0.2479 - accuracy: 0.6976

In [33]:
s5= define_model("s5")
s6= define_model("s6")
s7= define_model("s7")
s8= define_model("s8")

In [34]:
mm5_history=s5.fit(X_train, s5Train, batch_size=batch_size, epochs=10,verbose=1)
m6_history=s6.fit(X_train, s6Train, batch_size=batch_size, epochs=10,verbose=1)

Epoch 1/10
196/196 [==============================] - 1s 4ms/step - loss: 0.7175 - accuracy: 0.2392
Epoch 2/10
196/196 [==============================] - 1s 4ms/step - loss: 0.4100 - accuracy: 0.3035
Epoch 3/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3541 - accuracy: 0.3442
Epoch 4/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3337 - accuracy: 0.3554
Epoch 5/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3205 - accuracy: 0.3654
Epoch 6/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3133 - accuracy: 0.3735
Epoch 7/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3105 - accuracy: 0.3691
Epoch 8/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3102 - accuracy: 0.3697
Epoch 9/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3037 - accuracy: 0.3712
Epoch 10/10
196/196 [==============================] - 1s 4ms/step - loss: 0.3009 - accuracy: 0.3724

In [35]:
mm5_history=s7.fit(X_train, s7Train, batch_size=batch_size, epochs=10,verbose=1)
m6_history=s8.fit(X_train, s8Train, batch_size=batch_size, epochs=10,verbose=1)

Epoch 1/10
196/196 [==============================] - 1s 4ms/step - loss: 2.1296e-05 - accuracy: 0.9595
Epoch 2/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9966
Epoch 3/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9967
Epoch 4/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9966
Epoch 5/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9966
Epoch 6/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9967
Epoch 7/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9968
Epoch 8/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9966
Epoch 9/10
196/196 [==============================] - 1s 4ms/step - loss: 1.9085e-05 - accuracy: 0.9968
Epoch 10/10
196/196 [==============================] - 1s 4ms/st

In [36]:
o1=s1.get_layer("s1req").output
o2=s2.get_layer("s2req").output
o3=s3.get_layer("s3req").output
o4=s4.get_layer("s4req").output

o5=s5.get_layer("s5req").output
o6=s6.get_layer("s6req").output
o7=s7.get_layer("s7req").output
o8=s8.get_layer("s8req").output
out = tensorflow.keras.layers.concatenate([o1,o2,o3,o4, o5,o6,o7,o8])

output=Activation('relu')(out)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)
# output4=Activation('softmax')(output3)
mm=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,
          s3.get_layer("s3").input,s4.get_layer("s4").input,
          s5.get_layer("s5").input,s6.get_layer("s6").input,
          s7.get_layer("s7").input,s8.get_layer("s8").input], 
         output3)
my_weights=model.get_layer('d2').get_weights()
mm.get_layer('d11').set_weights(my_weights)

In [37]:
i=0
for l in mm.layers[:len(mm.layers)-1]:
    l.trainable=False

In [38]:
adam = Adam(0.0002,0.5,0.999)

mm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [39]:
batch_size = 128
mm_history=mm.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test], y_test))

Epoch 1/10
196/196 [==============================] - 5s 16ms/step - loss: 0.4481 - accuracy: 0.8348 - val_loss: 0.3788 - val_accuracy: 0.8374
Epoch 2/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3884 - accuracy: 0.8317 - val_loss: 0.3676 - val_accuracy: 0.8370
Epoch 3/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3790 - accuracy: 0.8329 - val_loss: 0.3671 - val_accuracy: 0.8376
Epoch 4/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3736 - accuracy: 0.8377 - val_loss: 0.3665 - val_accuracy: 0.8386
Epoch 5/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3736 - accuracy: 0.8387 - val_loss: 0.3666 - val_accuracy: 0.8390
Epoch 6/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3718 - accuracy: 0.8373 - val_loss: 0.3677 - val_accuracy: 0.8378
Epoch 7/10
196/196 [==============================] - 3s 14ms/step - loss: 0.3716 - accuracy: 0.8370 - val_loss: 0.3670 - val_accuracy: 0.8382

In [40]:
_, acc = mm.evaluate([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test], y_test, verbose=1)

782/782 [==============================] - 5s 6ms/step - loss: 0.3672 - accuracy: 0.8381


In [41]:
mm.summary()

Model: "model_38"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s3_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s4_input (InputLayer)           [(None, 500)]        0                                            
___________________________________________________________________________________________